# Using the RubinTV Mount Plotting to quantify mount errors

Craig Lage - 06-Oct-22

In [ ]:
import nest_asyncio
nest_asyncio.apply()
import sys, time, os, asyncio
from datetime import datetime
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from lsst.rubintv.production import mountTorques

from lsst_efd_client import EfdClient
from lsst.daf.butler import Butler
from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger

In [ ]:
client = EfdClient('summit_efd')
butler = Butler('/repo/LATISS', collections="LATISS/raw/all")
logger = DecoratedLogger.get_decorated_logger()

In [ ]:
figure = plt.figure(figsize=(16,22))
saveFilename = None
dayObss = [20220912, 20220913, 20220914, 20220927, 20220928, 20220929]
errs = []
for dayObs in dayObss:
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
        if record.observation_type not in ['bias', 'flat', 'dark']:
            try:
                exposureList.append([record.id, record.azimuth, 90.0-record.zenith_angle])
            except:
                continue
    for [expId, az, el] in exposureList:
        dataId = {'detector':0, 'exposure':expId}
        try:
            #print(expId)
            err = mountTorques.calculateMountErrors(dataId, butler, client, figure, saveFilename, logger)
            errs.append([expId, az, el, err[0], err[1], err[2]])
        except:
            print(f"expId {expId} Failed!")
            continue
errs = np.array(errs)
outfile = open(f'/home/craiglagegit/DATA/Temp_Tracking_Errors_202209.pkl', 'wb')
pkl.dump(errs,outfile)
outfile.close()

In [ ]:
# Get the pickled data
infile = open(f'/home/craiglagegit/DATA/Tracking_Errors_202209.pkl','rb')
errs = pkl.load(infile)
infile.close()
print(len(errs), "observations")

figure = plt.figure(figsize=(16,8))
plt.subplots_adjust(hspace = 0.5)
axes = ['Az', 'El']

for i, axis in enumerate(axes):
    axisErrs = errs[:,i+3]
    if i == 0:
        # Correct azimuth errors for cos(elevation)
        axisErrs *= np.cos(errs[:,2] * np.pi / 180.0)
    # Clip out values < 0 or > 1.0
    axisErrs = axisErrs[(axisErrs > 0.0) & (axisErrs < 1.0)]
    #axisErrs = np.clip(axisErrs, 0.0, 1.0)
    median = np.median(axisErrs)
    std = np.std(axisErrs)
    P75 = np.percentile(axisErrs, 75)
    P95 = np.percentile(axisErrs, 95)
    plt.subplot(1,2,i+1)
    plt.title(f"RMS Tracking error - 202209 - {axis}", fontsize = 18)
    plt.xlabel("RMS error - arcseconds", fontsize = 18)
    n, bins, patches = plt.hist(axisErrs, bins=50, range=(0,1.0))
    N = n.sum()
    y1 = n.max()*0.9
    step = y1 / 10.0
    y2 = y1 - step
    y3 = y2 - step
    y4 = y3 - step
    y5 = y4 - step
    plt.text(0.3, y1, f"N = {N:.0f} measurements", fontsize = 18)
    plt.text(0.3, y2, f"Median = {median:.2f}", fontsize = 18)
    plt.text(0.3, y3, f"Std = {std:.2f}", fontsize = 18)
    plt.text(0.3, y4, f"P75 = {P75:.2f}", fontsize = 18)
    plt.text(0.3, y5, f"P95 = {P95:.2f}", fontsize = 18)
plt.savefig("/home/craiglagegit/DATA/Tracking_Errors_202209.pdf")